# Notebook to transform OSeMOSYS output to same format as EGEDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from openpyxl import Workbook
import xlsxwriter
import pandas.io.formats.excel
import glob
import re

In [2]:

# Path for OSeMOSYS output

path_output = '../../data/3_OSeMOSYS_output'

# Path for OSeMOSYS to EGEDA mapping

path_mapping = '../../data/2_Mapping_and_other'

# Where to save finalised dataframe

path_final = '../../data/4_Joined'

In [3]:
# They're csv files so use a wild card (*) to grab the filenames

OSeMOSYS_filenames = glob.glob(path_output + "/*.xlsx")
OSeMOSYS_filenames

['../../data/3_OSeMOSYS_output\\02_BD_results_v1.1.xlsx',
 '../../data/3_OSeMOSYS_output\\04_CHL_results_v1.1.xlsx',
 '../../data/3_OSeMOSYS_output\\06_HKC_results.xlsx',
 '../../data/3_OSeMOSYS_output\\08_JPN_results_v1.0.xlsx',
 '../../data/3_OSeMOSYS_output\\09_ROK_results.xlsx',
 '../../data/3_OSeMOSYS_output\\11_MEX_results_v1.2.xlsx',
 '../../data/3_OSeMOSYS_output\\12_NZ_results_v1.1.xlsx',
 '../../data/3_OSeMOSYS_output\\13_PNG_results_v1.1.xlsx',
 '../../data/3_OSeMOSYS_output\\14_PE_results_v1.1.xlsx',
 '../../data/3_OSeMOSYS_output\\18_CT_results.xlsx']

In [4]:
# Read in mapping file

Mapping_file = pd.read_excel(path_mapping + '/OSeMOSYS mapping.xlsx', sheet_name = 'Mapping',  skiprows = 1)
Mapping_file = Mapping_file[Mapping_file['Balance'].isin(['TFC', 'TPES'])]

In [5]:
# Define unique workbook and sheet combinations

Unique_combo = Mapping_file.groupby(['Workbook', 'Sheet']).size().reset_index().loc[:, ['Workbook', 'Sheet']]

In [6]:
# Determine list of files to read based on the workbooks identified in the mapping file

file_df = pd.DataFrame()

for i in range(len(Unique_combo['Workbook'].unique())):
    _file = pd.DataFrame({'File': [entry for entry in OSeMOSYS_filenames if Unique_combo['Workbook'].unique()[i] in entry],
                         'Workbook': Unique_combo['Workbook'].unique()[i]})
    file_df = file_df.append(_file)

file_df = file_df.merge(Unique_combo, how = 'outer', on = 'Workbook')

In [7]:
# Create empty dataframe to store aggregated results 

aggregate_df1 = pd.DataFrame()

# Now read in the OSeMOSYS output files so that that they're all in one data frame (aggregate_df1)

for i in range(file_df.shape[0]):
    _df = pd.read_excel(file_df.iloc[i, 0], sheet_name = file_df.iloc[i, 2])
    _df['Workbook'] = file_df.iloc[i, 1]
    _df['Sheet'] = file_df.iloc[i, 2]
    aggregate_df1 = aggregate_df1.append(_df) 

interim_df1 = aggregate_df1[aggregate_df1['TIMESLICE'] != 'ONE']
interim_df2 = aggregate_df1[aggregate_df1['TIMESLICE'] == 'ONE']

interim_df1 = interim_df1.groupby(['TECHNOLOGY', 'FUEL', 'REGION', 'Workbook', 'Sheet']).sum().reset_index()

aggregate_df1 = interim_df2.append(interim_df1).reset_index(drop = True)


In [8]:
# Get maximum year column to build data frame below

year_columns = []

for item in list(aggregate_df1.columns):
    try:
        year_columns.append(int(item))
    except ValueError:
            pass

max_year = max(year_columns)

OSeMOSYS_years = list(range(2017, max_year + 1))

In [9]:
# fuel_code aggregations

# Start at lowest level first

gasoline_fuels = ['4_1_1_motor_gasoline', '4_1_2_aviation_gasoline']               
liquid_biofuels = ['9_8_1_biogasoline', '9_8_2_biodiesel', '9_8_4_other_liquid_biofuels']
other_renew = ['8_2_1_photovoltaic', '8_2_2_tide_wave_ocean', '8_2_3_wind', '8_2_4_solar']

# Then first level

coal_fuels = ['1_1_1_coking_coal', '1_x_coal_thermal', '1_3_lignite']
oil_fuels = ['3_1_crude_oil', '3_x_ngls']
petrol_fuels = ['4_1_gasoline', '4_2_naphtha', '4_3_jet_fuel', '4_4_other_kerosene', '4_5_gas_diesel_oil', 
                  '4_6_fuel_oil', '4_7_lpg', '4_8_refinery_gas_not_liq', '4_9_ethane', '4_10_other_petroleum_products']
gas_fuels = ['5_1_natural_gas', '5_2_lng']
renew_fuels = ['8_1_geothermal_power', '8_2_other_power', '8_3_geothermal_heat', '8_4_solar_heat']
other_fuels = ['9_1_fuel_wood_and_woodwaste', '9_2_bagasse', '9_3_charcoal', '9_4_other_biomass', '9_5_biogas', '9_6_industrial_waste', '9_7_municipal_solid_waste',
               '9_8_liquid_biofuels', '9_9_other_sources']

# Total

total_fuels = ['1_coal', '2_coal_products', '3_crude_oil_and_ngl', '4_petroleum_products', '5_gas', '6_hydro', '7_nuclear', '8_geothermal_solar_etc', '9_others',
               '10_electricity', '11_heat']

# total_renewables to be completed

###########################################################################################

# item_code_new aggregations

# Lowest level

industry_agg = ['13_1_iron_and_steel', '13_2_chemical_incl__petrochemical', '13_3_nonferrous_metals', '13_4_nonmetallic_mineral_products', 
                '13_5_transportation_equipment', '13_6_machinery', '13_7_mining_and_quarrying', '13_8_food_beverages_and_tobacco',
                '13_9_pulp_paper_and_printing', '13_10_wood_and_wood_products', '13_11_construction', '13_12_textiles_and_leather',
                '13_13_nonspecified_industry']

transport_agg = ['14_1_domestic_air_transport', '14_2_road', '14_3_rail', '14_4_domestic_water_transport', '14_5_pipeline_transport', '14_6_nonspecified_transport']

others_agg = ['15_1_1_commerce_and_public_services', '15_1_2_residential', '15_2_agriculture', '15_3_fishing', '15_4_nonspecified_others']

# Then first level

tpes_agg = ['1_indigenous_production', '2_imports', '3_exports', '4_1_international_marine_bunkers', '4_2_international_aviation_bunkers']

tfc_agg = ['13_industry_sector', '14_transport_sector', '15_other_sector', '16_nonenergy_use']

tfec_agg = ['13_industry_sector', '14_transport_sector', '15_other_sector']


In [10]:

# Now aggregate data based on the mapping

# That is group by REGION, TECHNOLOGY and FUEL

# First create empty dataframe

aggregate_df2 = pd.DataFrame()

# Then loop through based on different regions/economies and stitch back together

for region in aggregate_df1['REGION'].unique():
    interim_df1 = aggregate_df1[aggregate_df1['REGION'] == region]
    interim_df1 = interim_df1.merge(Mapping_file, how = 'left', on = ['TECHNOLOGY', 'FUEL'])
    interim_df1 = interim_df1.groupby(['item_code_new', 'fuel_code']).sum().reset_index()

    # Change export data to negative values
    exports_bunkers = interim_df1[interim_df1['item_code_new'].isin(['3_exports', '4_1_international_marine_bunkers', '4_2_international_aviation_bunkers'])]
    everything_else = interim_df1[~interim_df1['item_code_new'].isin(['3_exports', '4_1_international_marine_bunkers', '4_2_international_aviation_bunkers'])]

    s = exports_bunkers.select_dtypes(include=[np.number]) * -1
    exports_bunkers[s.columns] = s
    interim_df1 = exports_bunkers.append(everything_else)

    ########################### Aggregate fuel_code for new variables ###################################

    # Start with lowest level of aggregation (so that it can then be grabbed at higher levels of aggregation later)
    
    gasoline = interim_df1[interim_df1['fuel_code'].isin(gasoline_fuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '4_1_gasoline').reset_index()
    
    liquids = interim_df1[interim_df1['fuel_code'].isin(liquid_biofuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '9_8_liquid_biofuels').reset_index()

    renewables_others = interim_df1[interim_df1['fuel_code'].isin(other_renew)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '8_2_other_power').reset_index()

    interim_df2 = interim_df1.append([gasoline, liquids, renewables_others]).reset_index(drop = True)

    # Now first level fuels

    coal = interim_df2[interim_df2['fuel_code'].isin(coal_fuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '1_coal').reset_index()

    oil = interim_df2[interim_df2['fuel_code'].isin(oil_fuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '3_crude_oil_and_ngl').reset_index()

    petrol = interim_df2[interim_df2['fuel_code'].isin(petrol_fuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '4_petroleum_products').reset_index()

    gas = interim_df2[interim_df2['fuel_code'].isin(gas_fuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '5_gas').reset_index()

    renew = interim_df2[interim_df2['fuel_code'].isin(renew_fuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '8_geothermal_solar_etc').reset_index()

    others = interim_df2[interim_df2['fuel_code'].isin(other_fuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '9_others').reset_index()

    interim_df3 = interim_df2.append([coal, oil, petrol, gas, renew, others]).reset_index(drop = True)

    # And total fuels

    total_f = interim_df3[interim_df3['fuel_code'].isin(total_fuels)].groupby(['item_code_new'])\
        .sum().assign(fuel_code = '12_total').reset_index()

    interim_df4 = interim_df3.append(total_f).reset_index(drop = True)

    ################################ And now item_code_new ######################################

    # Start with lowest level

    industry = interim_df4[interim_df4['item_code_new'].isin(industry_agg)].groupby(['fuel_code'])\
        .sum().assign(item_code_new = '13_industry_sector').reset_index()

    transport = interim_df4[interim_df4['item_code_new'].isin(transport_agg)].groupby(['fuel_code'])\
        .sum().assign(item_code_new = '14_transport_sector').reset_index()

    bld_ag_other = interim_df4[interim_df4['item_code_new'].isin(others_agg)].groupby(['fuel_code'])\
        .sum().assign(item_code_new = '15_other_sector').reset_index()

    interim_df5 = interim_df4.append([industry, transport, bld_ag_other]).reset_index(drop = True)

    # Now higher level agg

    #Might need to check this depending on whether exports is negative
    tpes = interim_df5[interim_df5['item_code_new'].isin(tpes_agg)].groupby(['fuel_code'])\
        .sum().assign(item_code_new = '6_total_primary_energy_supply').reset_index()

    tfc = interim_df5[interim_df5['item_code_new'].isin(tfc_agg)].groupby(['fuel_code'])\
        .sum().assign(item_code_new = '11_total_final_consumption').reset_index()

    tfec = interim_df5[interim_df5['item_code_new'].isin(tfec_agg)].groupby(['fuel_code'])\
        .sum().assign(item_code_new = '12_total_final_energy_consumption').reset_index()

    interim_df6 = interim_df5.append([tpes, tfc, tfec]).reset_index(drop = True)

    # Now add in economy reference
    interim_df6['economy'] = region

    # Now append economy dataframe to communal data frame 
    aggregate_df2 = aggregate_df2.append(interim_df6)
    

aggregate_df2 = aggregate_df2[['economy', 'fuel_code', 'item_code_new'] + OSeMOSYS_years]
aggregate_df2

,economy,fuel_code,item_code_new,2017,2018,2019,2020,2021,2022,2023,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,02_BD,2_coal_products,3_exports,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,...,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08
1,02_BD,3_1_crude_oil,3_exports,-2.191779e+02,-2.101628e+02,-2.303920e+02,-2.303920e+02,-2.102988e+02,-1.818580e+02,-1.813966e+02,...,-2.361202e+02,-2.352332e+02,-2.343462e+02,-2.334591e+02,-2.316851e+02,-2.307980e+02,-2.299110e+02,-2.281369e+02,-2.272499e+02,-2.263629e+02
2,02_BD,3_x_ngls,3_exports,-2.249572e+01,-1.807542e+01,-2.030860e+01,-2.030860e+01,-1.197997e+01,-1.912790e-01,-0.000000e+00,...,-2.976500e-01,-4.005991e-01,-1.708660e-01,-4.417411e-02,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
3,02_BD,4_1_1_motor_gasoline,3_exports,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-1.008092e+01,-1.008092e+01,...,-1.139814e+02,-1.141465e+02,-1.143218e+02,-1.145083e+02,-1.150255e+02,-1.151397e+02,-1.153951e+02,-1.158571e+02,-1.162655e+02,-1.164235e+02
4,02_BD,4_3_jet_fuel,3_exports,-0.000000e+00,-7.044757e-01,-7.044757e-01,-7.044757e-01,-5.318031e+00,-1.184825e+01,-1.195421e+01,...,-8.033945e+01,-8.054312e+01,-8.074679e+01,-8.095046e+01,-8.135780e+01,-8.156147e+01,-8.176514e+01,-8.217248e+01,-8.237615e+01,-8.257982e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,14_PE,9_1_fuel_wood_and_woodwaste,6_total_primary_energy_supply,1.258483e+01,1.226243e+01,1.156242e+01,9.368042e+00,9.471256e+00,9.313503e+00,9.219410e+00,...,5.731900e+00,5.573615e+00,5.432999e+00,5.256254e+00,5.067047e+00,4.833604e+00,4.606275e+00,4.368048e+00,4.118551e+00,3.870065e+00
60,14_PE,9_2_bagasse,6_total_primary_energy_supply,6.072373e+00,5.916810e+00,5.579044e+00,4.520223e+00,4.570026e+00,4.493908e+00,4.448506e+00,...,2.765730e+00,2.689355e+00,2.621505e+00,2.536223e+00,2.444928e+00,2.332288e+00,2.222598e+00,2.107650e+00,1.987264e+00,1.867365e+00
61,14_PE,9_3_charcoal,6_total_primary_energy_supply,1.230538e+00,1.199014e+00,1.130567e+00,9.160019e-01,9.260942e-01,9.106692e-01,9.014688e-01,...,5.604621e-01,5.449850e-01,5.312356e-01,5.139536e-01,4.954531e-01,4.726271e-01,4.503990e-01,4.271054e-01,4.027096e-01,3.784128e-01
62,14_PE,9_4_other_biomass,6_total_primary_energy_supply,8.971582e+01,8.971582e+01,8.971582e+01,8.971582e+01,8.971582e+01,8.971582e+01,8.971582e+01,...,8.932119e+01,8.921922e+01,8.971582e+01,8.971582e+01,1.042637e+02,1.255633e+02,1.479801e+02,1.710830e+02,1.950606e+02,2.200965e+02


In [11]:
# Now load the EGEDA_years data frame

EGEDA_years = pd.read_csv('../../data/1_EGEDA/EGEDA_2020_June_22_wide_years_PJ.csv')

In [12]:
# Remove 2017 which is already in the EGEDA historical

aggregate_df2_tojoin = aggregate_df2[['economy', 'fuel_code', 'item_code_new'] + OSeMOSYS_years[1:]]

In [13]:
# Join EGEDA historical to OSeMOSYS results

Joined_df = EGEDA_years.merge(aggregate_df2_tojoin, on = ['economy', 'fuel_code', 'item_code_new'], how = 'left')
Joined_df.to_csv(path_final + '/OSeMOSYS_to_EGEDA.csv', index = False)